# USA Housing

Vamos analisar o conjunto de dados USA Housing e tentar prever o valor das casas. Aqui vou focar apenas na construção do modelo com Random Forests, e não farei análises muito aprofundadas.

In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import sklearn.metrics as metrics

import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('USA_Housing.csv') #Lendo os dados

In [3]:
df.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address
0,79545.458574,5.682861,7.009188,4.09,23086.800503,1.059034e+06,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701..."
1,79248.642455,6.002900,6.730821,3.09,40173.072174,1.505891e+06,"188 Johnson Views Suite 079\nLake Kathleen, CA..."
2,61287.067179,5.865890,8.512727,5.13,36882.159400,1.058988e+06,"9127 Elizabeth Stravenue\nDanieltown, WI 06482..."
3,63345.240046,7.188236,5.586729,3.26,34310.242831,1.260617e+06,USS Barnett\nFPO AP 44820
4,59982.197226,5.040555,7.839388,4.23,26354.109472,6.309435e+05,USNS Raymond\nFPO AE 09386


Todas as colunas são numéricas, exceto a coluna 'Address'. Analisando os dados da coluna, podemos ver que todas contém a sigla Estado em que a casa se localiza, e por isso iremos tentar extrair essa variável.

Em um primeiro momento, pode-se utilizar a estratégia de fazer um [str.split](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html) no char ',', mas isso não funcionará para todos os dados. Podemos ver, por exemplo, que as linhas 3 e 4 possuem a string 'FPO' antes da sigla do estado. Vamos então separar no espaço ' ' e pegar o segundo elemento da lista de trás pra frente, criando uma nova coluna chamada 'State', e depois excluir a coluna 'Address'.

In [99]:
df['State'] = df['Address'].apply(lambda X: X.split(' ')[-2])

df.drop('Address', axis=1, inplace=True)

df['State'].unique()

array(['NE', 'CA', 'WI', 'AP', 'AE', 'KS', 'CO', 'TN', 'AA', 'NM', 'PW',
       'AR', 'HI', 'ME', 'IN', 'MI', 'DE', 'AZ', 'MA', 'MN', 'AL', 'NY',
       'NV', 'VA', 'ID', 'OK', 'NH', 'MO', 'WV', 'WY', 'MH', 'UT', 'SD',
       'CT', 'AK', 'WA', 'RI', 'NJ', 'KY', 'NC', 'IA', 'VT', 'FM', 'ND',
       'LA', 'MP', 'OR', 'TX', 'DC', 'PR', 'MT', 'AS', 'OH', 'MS', 'IL',
       'VI', 'GA', 'PA', 'MD', 'SC', 'GU', 'FL'], dtype=object)

Agora temos uma coluna com os nomes dos Estados.

In [100]:
X = df.drop('Price', axis=1)
y = df[['Price']]

In [101]:
X, X_test, y, y_test = train_test_split(X, y, test_size=0.3, random_state=101) #Separando 30% dos dados para o teste.

Depois de feito o split treino e teste, vamos aplicar as transformações nos dados. Precisamos converter a coluna 'State' para numérico.

In [102]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

X['state_code'] = le.fit_transform(X['State'])
X_test['state_code'] = le.fit_transform(X_test['State'])

X.drop('State', axis=1, inplace=True)
X_test.drop('State', axis=1, inplace=True)

In [113]:
reg = RandomForestRegressor()

reg.fit(X, y)
y_pred = reg.predict(X_test)

print(np.sqrt(metrics.mean_absolute_error(y_pred, y_test)))

313.3490204966272


311.7348585488764


In [108]:
reg.feature_names_in_

array(['Avg. Area Income', 'Avg. Area House Age',
       'Avg. Area Number of Rooms', 'Avg. Area Number of Bedrooms',
       'Area Population', 'state_code'], dtype=object)

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,state_code
2654,86690.873301,6.259901,6.676265,3.23,42589.624391,27
2468,59866.947700,5.870330,5.899076,4.16,32064.597156,11
290,74372.138452,6.562380,8.184511,6.35,34321.960155,56
1463,61370.323490,6.529605,6.606744,4.30,20600.511000,20
4508,52652.652336,5.688943,7.217268,4.06,34776.585907,33
...,...,...,...,...,...,...
4171,56610.642563,4.846832,7.558137,3.29,25494.740298,17
599,70596.850945,6.548274,6.539986,3.10,51614.830136,58
1361,55621.899104,3.735942,6.868291,2.30,63184.613147,12
1547,63044.460096,5.935261,5.913454,4.10,32725.279544,36
